From https://turi.com/learn/userguide/deployment/pipeline-example.html

In [14]:
import graphlab as gl

In [15]:
def clean_file(path):
    """
    Takes a CSV file passed in as a param and cleans it into an SFrame.
    In particular, it parses and drops None values.
    """
    import graphlab as gl

    sf = gl.SFrame.read_csv(path, delimiter='\t')
    sf = sf.dropna()
    return sf

In [16]:
def train_model(data):
    """
    Takes an SFrame as input and uses it to train a model,
    setting the train and test sets as outputs along with the trained
    model.
    """
    import graphlab as gl

    model = gl.recommender.create(data, user_id='user',
                                  item_id='movie',
                                  target='rating')
    return model

In [17]:
def recommend_items(model, data):
    recommendations = model.recommend(users=data['user'])
    return recommendations

In [18]:
def my_workflow(path):
    # Clean file
    data = clean_file(path)

    # Train model.
    model = train_model(data)

    # Make recommendations.
    recommendations = recommend_items(model, data)

    # Return the SFrame of recommendations.
    return recommendations

In [20]:
ec2config = gl.deploy.Ec2Config(instance_type='m3.xlarge',
                                aws_access_key_id='xxxxx',
                                aws_secret_access_key='xxxxx')

ec2 = gl.deploy.ec2_cluster.create(name='ec2',
                                   s3_path='s3://graphlab-deployment-smu/test',
                                   ec2_config=ec2config,
                                  idle_shutdown_timeout=1000000)

job_ec2 = gl.deploy.job.create(my_workflow,
        environment=ec2,
        path='https://static.turi.com/datasets/movie_ratings/sample.large')

# get the results
job_ec2.get_results()

[INFO] graphlab.connect.aws._ec2: Launching an m3.xlarge instance in the us-west-2c availability zone, with id: i-070fb740926694ee4. You will be responsible for the cost of this instance.
[INFO] graphlab.deploy._executionenvironment: Waiting for i-070fb740926694ee4 to start up.


RuntimeError: Unable to start host(s). Please terminate manually from the AWS console.